In [ ]:
!pip install -U langchain langchain-core langchain-community langchain-google-genai langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
from langchain_google_genai import ChatGoogleGenerativeAI
# 코랩 Secrets에서 GOOGLE_API_KEY를 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


In [ ]:
#데이터베이스 저장된 대화 기록 확인
import sqlite3
import json # 'message' 컬럼의 JSON 문자열을 파싱하기 위함
def get_history_from_db(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  conn = sqlite3.connect(db_path)
  cursor = conn.cursor()
  print(f"데이터베이스 '{db_path}'에 성공적으로 연결되었습니다.")

  cursor.execute("SELECT * FROM message_store")
  print("\n대화 기록")
  rows = cursor.fetchall()
  for row in rows:
    msg_id, session_id, message_json = row
    try:
        # 'message' 컬럼은 JSON 문자열이므로 파싱
        message_data = json.loads(message_json)
        msg_type = message_data.get("type", "unknown")

        # LangChain 메시지의 실제 내용은 'data' 딕셔너리 안에 'content' 키로 저장
        msg_content = message_data.get("data", {}).get("content", "[내용 없음]")

        print(f"ID: {msg_id}, Session: {session_id}")
        print(f"  Type: {msg_type.upper()}")
        print(f"  Content: {msg_content}")
        print("-" * 40)
    except json.JSONDecodeError:
        print(f"ID: {msg_id}, Session: {session_id}, Message: {message_json[:100]}...") # JSON 파싱 오류 시 원본 출력
        print("-" * 40)

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
store_histories = {} # SQLChatMessageHistory 객체를 저장
# 데이터베이스에서 대화내용 가져오는 함수
def get_chat_history(session_id):
  db_path = f"/content/drive/MyDrive/LangChain/{session_id}_chat.db"
  if session_id not in store_histories:
      print(f"{session_id} 에 대한 SQLChatMessageHistory 생성 및 연결")
      sql_chat_history = SQLChatMessageHistory(
            session_id=session_id,
            connection=f"sqlite:///{db_path}"
      )
      store_histories[session_id] = sql_chat_history
  return store_histories[session_id] # SQLChatMessageHistory 객체 반환

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 한국어 챗봇입니다."),
    MessagesPlaceholder(variable_name="history"), # 현재 대화 기록
    ("human", "{input}"),
])

#ConversationChain 사용

## RunnableWithMessageHistory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
# 대화 요약
store_summarizers = {} # ConversationSummaryBufferMemory 객체 저장 (요약용)
def get_session_summarizer(session_id):
    if session_id not in store_summarizers:
        print(f"새로운 세션 ID: {session_id} 에 대한 ConversationSummaryBufferMemory (요약용) 생성")

        memory = ConversationSummaryBufferMemory(
            llm=llm,
            max_token_limit=200, # 요약에 사용할 최대 토큰 제한
            memory_key="history", # 이 메모리에서 대화 기록을 가져올 키
            return_messages=True # 메시지 객체 리스트 형태로 반환
        )

        store_summarizers[session_id]  = ConversationChain(
          llm = llm,
          # ConversationSummaryBufferMemory 사용
          memory=memory,
          prompt=prompt,
        )
    return store_summarizers[session_id]

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  conversation_chain = get_session_summarizer(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      conversation_chain_with_history = RunnableWithMessageHistory(
        conversation_chain, # conversationChain
        get_chat_history,
        input_messages_key="input",
        history_messages_key="history",
      )
      config = {"configurable": {"session_id": session_id}}
      inputs = {"input": user_input}
      response = conversation_chain_with_history.invoke(inputs, config)['response']
      print(response)
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션1
session_id = 'session1_s'
chatbot(session_id)

<ipython-input-7-3742732623>:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(
<ipython-input-7-3742732623>:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  store_summarizers[session_id]  = ConversationChain(



--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session1_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 서울에 대해 간략하게 설명해줘
챗봇 (답변 생성 중)...
session1_s 에 대한 SQLChatMessageHistory 생성 및 연결
서울은 대한민국의 수도이자 최대 도시로, 역사와 현대가 공존하는 매력적인 도시입니다.  한강을 중심으로 발전해왔으며, 고궁과 같은 전통 건축물부터 초고층 빌딩과 최첨단 시설까지 다양한 모습을 볼 수 있습니다.  세계적인 문화, 예술, 경제 중심지이기도 하며, 풍부한 역사 유적과 활기찬 현대 문화를 동시에 경험할 수 있는 곳입니다.  짧게 말해, 서울은 역동적이고 다채로운 매력을 가진 도시입니다.

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 내 이름은 테디야
챗봇 (답변 생성 중)...
반갑습니다, 테디님!  무엇을 도와드릴까요?

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#세션2
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...
session2_s 에 대한 SQLChatMessageHistory 생성 및 연결
아니요, 저는 대화형 인공지능 모델이기 때문에, 이전 대화를 기억하지 못합니다.  처음 만난 것처럼 생각해 주세요.  혹시 이름을 알려주시겠어요? 😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 내 이름은 디테야
챗봇 (답변 생성 중)...
반갑습니다, 디테님!  무엇을 도와드릴까요?  😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 제주도에 대해 간략하게 알려줄래?
챗봇 (답변 생성 중)...
제주도는 대한민국 남쪽에 위치한 화산섬으로, 아름다운 자연경관과 독특한 문화를 자랑하는 곳입니다.  한라산이라는 휴화산을 중심으로 섬 전체가 유네스코 세계자연유산으로 등재되어 있으며,  다양한 볼거리와 즐길 거리가 풍부합니다.  

주요 특징으로는:

* **아름다운 자연:** 푸른 바다, 검은 모래 해변, 울창한 숲, 그리고 한라산의 장엄한 풍경 등 다채로운 자연을 경험할 수 있습니다.
* **다양한 관광지:** 성산일출봉, 만장굴, 섭지코지, 정방폭포 등 많은 관광 명소가 있습니다.
* **독특한 문화:** 제주도만의 독특한 언어와 음식, 그리고 전통문화를 체험할 수 있습니다.  돌하르방, 제주 돌담 등도 유명합니다.
* **다양한 액티비티:** 해양 액티비티, 트레킹, 승마 등 다양한 즐길 거리가 있습니다.


간략하게 설명드렸지만, 제주도는 훨씬 더 많은 매력을 가지고 있습니다.  어떤 부분에 대해 더 자세히 알고 싶으신가요?  혹은 어떤 종류의 여행을 계획하고 계신가요?  좀 더 구체적으로 말씀해주시

In [ ]:
#세션1
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 내 이름을 기억해?
챗봇 (답변 생성 중)...
네, 테디님.  이름 기억하고 있습니다!  무엇을 도와드릴까요?

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#세션1 메모리 확인
summarizer_memory = get_session_summarizer(session_id)
summarizer_memory.memory.load_memory_variables({})['history']

[SystemMessage(content="The AI describes Seoul as the capital and largest city of South Korea, a captivating blend of history and modernity.  It's developed around the Han River and features a mix of traditional architecture like palaces and modern skyscrapers and cutting-edge facilities.  The AI highlights Seoul as a global center for culture, arts, and economics, offering a unique experience of rich historical sites and vibrant contemporary culture, ultimately characterizing it as a dynamic and diverse city.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름은 테디야', additional_kwargs={}, response_metadata={}),
 AIMessage(content='반갑습니다, 테디님!  무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름을 기억해?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='네, 테디님.  이름 기억하고 있습니다!  무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})]

In [ ]:
#세션1 데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session1_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session1_s
  Type: HUMAN
  Content: 서울에 대해 간략하게 설명해줘
----------------------------------------
ID: 2, Session: session1_s
  Type: AI
  Content: 서울은 대한민국의 수도이자 최대 도시로, 역사와 현대가 공존하는 매력적인 도시입니다.  한강을 중심으로 발전해왔으며, 고궁과 같은 전통 건축물부터 초고층 빌딩과 최첨단 시설까지 다양한 모습을 볼 수 있습니다.  세계적인 문화, 예술, 경제 중심지이기도 하며, 풍부한 역사 유적과 활기찬 현대 문화를 동시에 경험할 수 있는 곳입니다.  짧게 말해, 서울은 역동적이고 다채로운 매력을 가진 도시입니다.
----------------------------------------
ID: 3, Session: session1_s
  Type: HUMAN
  Content: 내 이름은 테디야
----------------------------------------
ID: 4, Session: session1_s
  Type: AI
  Content: 반갑습니다, 테디님!  무엇을 도와드릴까요?
----------------------------------------
ID: 5, Session: session1_s
  Type: HUMAN
  Content: 내 이름을 기억해?
----------------------------------------
ID: 6, Session: session1_s
  Type: AI
  Content: 네, 테디님.  이름 기억하고 있습니다!  무엇을 도와드릴까요?
----------------------------------------


##SQLChatMessageHistory 직접 사용

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  sql_chat_history = get_chat_history(session_id)
  conversation_chain = get_session_summarizer(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      response = conversation_chain.invoke({"input":user_input})['response']
      print(response)

      # 사용자 메시지 추가
      sql_chat_history.add_user_message(
        user_input
      )

      # AI 메시지 추가
      sql_chat_history.add_ai_message(response)
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션1
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 지하철에 대해 간략히 설명해줄래?
챗봇 (답변 생성 중)...
네, 테디님! 서울 지하철은 매우 편리하고 효율적인 대중교통 수단입니다.  전철이라고도 불리는 서울 지하철은 서울 전역을 아우르는 광범위한 노선망을 가지고 있어, 거의 모든 곳으로 편리하게 이동할 수 있습니다.  각 노선은 색깔로 구분되어 있어서 찾기 쉽고, 역마다 안내 표지판과 영어 안내 방송도 잘 되어 있어 외국인도 이용하기 편리합니다.  Tmoney 카드나 교통카드를 이용하면 편리하게 이용할 수 있으며,  시간대에 따라 다소 혼잡할 수 있지만, 전반적으로 빠르고 안전하게 목적지까지 이동할 수 있는 좋은 교통 수단입니다.  궁금한 점이 더 있으시면 언제든지 물어보세요!

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 고마워
챗봇 (답변 생성 중)...
천만에요!  무엇을 도와드릴까요?  혹시 서울에 대해 더 궁금한 점이 있으신가요?  제가 아는 한도 내에서 최선을 다해 도와드리겠습니다. 😊

--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: quit
챗봇을 종료합니다.


In [ ]:
#메모리 확인
summarizer_memory = get_session_summarizer(session_id)
summarizer_memory.memory.load_memory_variables({})['history']

[SystemMessage(content="The AI describes Seoul as the capital and largest city of South Korea, a captivating blend of history and modernity, developed around the Han River and featuring a mix of traditional architecture and modern skyscrapers.  It's a global center for culture, arts, and economics, offering a unique experience of rich historical sites and vibrant contemporary culture, ultimately characterized as a dynamic and diverse city.  The AI then interacts with a user named Teddy, remembering the name and offering assistance.  Finally, the AI provides a brief description of Seoul's subway system, highlighting its convenience, efficiency, extensive network, clear signage (including English), and the ease of using Tmoney cards.  The AI notes that while it can be crowded at times, it's generally a fast and safe way to travel around the city.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='고마워', additional_kwargs={}, response_metadata={}),
 AIMessage(content='천만

In [ ]:
#데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session1_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session1_s
  Type: HUMAN
  Content: 서울에 대해 간략하게 설명해줘
----------------------------------------
ID: 2, Session: session1_s
  Type: AI
  Content: 서울은 대한민국의 수도이자 최대 도시로, 역사와 현대가 공존하는 매력적인 도시입니다.  한강을 중심으로 발전해왔으며, 고궁과 같은 전통 건축물부터 초고층 빌딩과 최첨단 시설까지 다양한 모습을 볼 수 있습니다.  세계적인 문화, 예술, 경제 중심지이기도 하며, 풍부한 역사 유적과 활기찬 현대 문화를 동시에 경험할 수 있는 곳입니다.  짧게 말해, 서울은 역동적이고 다채로운 매력을 가진 도시입니다.
----------------------------------------
ID: 3, Session: session1_s
  Type: HUMAN
  Content: 내 이름은 테디야
----------------------------------------
ID: 4, Session: session1_s
  Type: AI
  Content: 반갑습니다, 테디님!  무엇을 도와드릴까요?
----------------------------------------
ID: 5, Session: session1_s
  Type: HUMAN
  Content: 내 이름을 기억해?
----------------------------------------
ID: 6, Session: session1_s
  Type: AI
  Content: 네, 테디님.  이름 기억하고 있습니다!  무엇을 도와드릴까요?
----------------------------------------
ID: 7, Session: session1_s
  Type: HUMAN
  Co

In [ ]:
#세션3
session_id = 'session3_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
session3_s 에 대한 SQLChatMessageHistory 생성 및 연결
새로운 세션 ID: session3_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 한국의 기후에 대해 설명해줘
챗봇 (답변 생성 중)...
안녕하세요! 한국의 기후에 대해 설명해 드릴게요.  간단히 말씀드리면 한국은 **온대 계절풍 기후**에 속합니다.  하지만 지역에 따라 기후 차이가 상당히 크다는 점을 꼭 기억해주세요.  자세히 설명드리면 다음과 같습니다.

**1. 계절의 변화가 뚜렷한 온대 기후:** 사계절이 뚜렷하게 나타납니다. 봄에는 따뜻하고 습하며, 여름에는 덥고 습하고, 가을에는 시원하고 건조하며, 겨울에는 춥고 건조합니다.  특히, 여름과 겨울의 기온 차이가 매우 큽니다.

**2. 계절풍의 영향:**  계절풍의 영향을 크게 받습니다. 여름에는 남쪽에서 습한 해양성 계절풍이 불어와 무덥고 비가 많이 오며, 겨울에는 북쪽에서 차고 건조한 대륙성 계절풍이 불어와 춥고 건조합니다.  이 계절풍의 영향으로 여름에는 장마철이, 겨울에는 건조한 날씨가 지속됩니다.

**3. 지역별 기후 차이:**  한국의 지형적 특징 때문에 지역별 기후 차이가 매우 큽니다.
    * **서해안:** 여름철에 습도가 높고 강수량이 많으며, 겨울철에는 서해안에 위치한 섬들의 영향으로 눈이 적게 내립니다.  또한, 해풍의 영향으로 기온 변화가 완만합니다.
    * **동해안:** 겨울철에 강한 북서풍의 영향으로 강설량이 많고, 여름철에는 동해안을 따라 흐르는 냉수대의 영향으로 서해안보다 기온이 낮습니다.
    * **내륙:** 여름에는 낮과 밤의 기온 차가 크고, 겨울에는 매우 춥습니다. 강수량은 서해안이나 동해안보다 적습니다.
    * **남부지방:**  온화한 기후를 보이며, 겨울에도 눈이 적게 내립니다.  여

In [ ]:
#메모리 확인
summarizer_memory = get_session_summarizer(session_id)
summarizer_memory.memory.load_memory_variables({})['history']

[SystemMessage(content="The human asks the AI to describe the climate of Korea. The AI responds that Korea has a temperate monsoon climate, but with significant regional variations.  It details four key aspects: 1) distinct four seasons with large temperature differences between summer and winter; 2) the significant influence of monsoons, bringing humid, rainy summers and cold, dry winters; 3) substantial regional differences, including humid summers and less snow on the west coast, heavy snowfall and cooler summers on the east coast, large temperature differences between day and night in inland areas, milder winters and frequent heatwaves in the south, and long, cold winters and short, cool summers in the north; and 4) the increasing impact of climate change, leading to more frequent extreme weather events like heatwaves, heavy rains, and droughts.  The AI concludes that while Korea's climate is generally temperate monsoon, regional variations must be considered when planning.", addit

In [ ]:
#데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session3_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session3_s
  Type: HUMAN
  Content: 한국의 기후에 대해 설명해줘
----------------------------------------
ID: 2, Session: session3_s
  Type: AI
  Content: 안녕하세요! 한국의 기후에 대해 설명해 드릴게요.  간단히 말씀드리면 한국은 **온대 계절풍 기후**에 속합니다.  하지만 지역에 따라 기후 차이가 상당히 크다는 점을 꼭 기억해주세요.  자세히 설명드리면 다음과 같습니다.

**1. 계절의 변화가 뚜렷한 온대 기후:** 사계절이 뚜렷하게 나타납니다. 봄에는 따뜻하고 습하며, 여름에는 덥고 습하고, 가을에는 시원하고 건조하며, 겨울에는 춥고 건조합니다.  특히, 여름과 겨울의 기온 차이가 매우 큽니다.

**2. 계절풍의 영향:**  계절풍의 영향을 크게 받습니다. 여름에는 남쪽에서 습한 해양성 계절풍이 불어와 무덥고 비가 많이 오며, 겨울에는 북쪽에서 차고 건조한 대륙성 계절풍이 불어와 춥고 건조합니다.  이 계절풍의 영향으로 여름에는 장마철이, 겨울에는 건조한 날씨가 지속됩니다.

**3. 지역별 기후 차이:**  한국의 지형적 특징 때문에 지역별 기후 차이가 매우 큽니다.
    * **서해안:** 여름철에 습도가 높고 강수량이 많으며, 겨울철에는 서해안에 위치한 섬들의 영향으로 눈이 적게 내립니다.  또한, 해풍의 영향으로 기온 변화가 완만합니다.
    * **동해안:** 겨울철에 강한 북서풍의 영향으로 강설량이 많고, 여름철에는 동해안을 따라 흐르는 냉수대의 영향으로 서해안보다 기온이 낮습니다.
    * **내륙:** 여름에는 낮과 밤의 기온 차가 크고, 겨울에는 매우 춥습니다. 강수량은 서해안이나 동해안보다 적습니다.
    * **남부지방:**  온화한 기후를 보이며, 

#메모리 직접 사용

##RunnableWithMessageHistory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
# 대화 요약
store_summarizers = {} # ConversationSummaryBufferMemory 객체 저장 (요약용)
def get_session_summarizer(session_id):
    if session_id not in store_summarizers:
        print(f"새로운 세션 ID: {session_id} 에 대한 ConversationSummaryBufferMemory (요약용) 생성")

        store_summarizers[session_id] = ConversationSummaryBufferMemory(
            llm=llm,
            max_token_limit=200, # 요약에 사용할 최대 토큰 제한
            memory_key="history", # 이 메모리에서 대화 기록을 가져올 키
            return_messages=True # 메시지 객체 리스트 형태로 반환
        )

    return store_summarizers[session_id]

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  memory = get_session_summarizer(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    # config 설정. 세션 ID 기준으로 대화 기록
    config = {"configurable": {"session_id": session_id}}
    try:
      # ConversationSummaryBufferMemory에서 요약된 내용 로드
      history = memory.load_memory_variables({})['history']
      print("\n--- history에 저장된 내용 ---")
      print(history)
      print("---history 내용 출력 끝---\n")
      print("챗봇 (답변 생성 중)...")
      config = {"configurable": {"session_id": session_id}}
      inputs = {"input": user_input, "history":history}
      chain_with_history = RunnableWithMessageHistory(
        prompt | llm,
        get_chat_history,
        input_messages_key="input",
        history_messages_key="history",
      )
      response = chain_with_history.invoke(inputs, config).content
      print(response)
      print()
      # 사용자 입력과 AI의 답변을 ConversationSummaryMemory에 전달하여 요약 업데이트
      memory.save_context(
                {"input": user_input}, # 사용자 입력
                {"output": response} # AI의 답변
      )
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션3. 메모리는 초기화됐지만 데이터베이스에 저장된 내용을 바탕으로 잘 대답함.
session_id = 'session3_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session3_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 이번엔 지형에 대해 설명해줄래?

--- history에 저장된 내용 ---
[]
---history 내용 출력 끝---

챗봇 (답변 생성 중)...
알겠습니다! 한국의 지형에 대해 설명해 드릴게요.  한국은 국토 면적이 작지만, 다양하고 복잡한 지형을 가지고 있어 '산지의 나라'라고 불리기도 합니다.  크게 세 가지로 나누어 설명해 드릴게요.

**1. 산지:** 한국 국토의 약 70%를 차지하는 산지가 가장 큰 특징입니다.  태백산맥을 중심으로 동쪽과 서쪽으로 여러 산맥이 뻗어 있습니다.

* **태백산맥:**  한국의 등뼈라고 불리는 태백산맥은 강원도를 중심으로 남북으로 길게 뻗어 있으며, 높고 험준한 산들이 많습니다.  설악산, 오대산, 태백산 등 유명한 산들이 이 산맥에 속합니다.
* **소백산맥:** 태백산맥에서 남쪽으로 뻗어 나온 산맥으로, 충청북도와 경상북도에 걸쳐 있습니다. 소백산, 속리산 등이 이 산맥에 속합니다.
* **노령산맥:** 전라북도와 전라남도에 걸쳐 있는 산맥으로, 비교적 낮고 완만한 산들이 많습니다.
* **차령산맥:** 충청남도와 경기도에 걸쳐 있는 산맥으로, 비교적 낮은 산들이 많습니다.  
* **함경산맥:** 북한 지역에 위치한 산맥으로, 높고 험준한 산들이 많습니다. 백두산이 이 산맥에 속합니다.

이러한 산맥들은 서로 연결되어 복잡한 산지 지형을 이루고 있으며,  산지 사이에는 분지와 골짜기가 발달해 있습니다.

**2. 평야:** 산지 사이에 좁게 분포하는 평야는 농업에 중요한 역할을 합니다.  주요 평야로는 다음과 같은 곳들이 있습니다.

* **김해평야:** 경상남도 김해 지역에 위치한 비옥한 평야입니다.
* **호남평야:** 전라남도와 전라북도에 걸쳐 있

In [ ]:
#메모리 확인
memory = get_session_summarizer(session_id)
memory.load_memory_variables({})['history']

[SystemMessage(content='The human asks the AI to describe the geography of Korea. The AI explains that Korea, despite its small land area, has diverse and complex terrain, often called a "mountainous country."  It categorizes the geography into three main parts: mountains (approximately 70% of the land, including the Taebaek, Sobaek, Noryeong, Charyeong, and Hamgyeong mountain ranges, with various notable peaks), plains (smaller areas crucial for agriculture, such as the Gimhae, Honam, Nonsan, Seosan, and Hwanghae plains), and coastlines (divided into the East, West, and South coasts, each with distinct characteristics like the East Coast\'s relatively simple coastline and the West Coast\'s extensive tidal flats).  The AI concludes that Korea\'s geography—predominantly mountainous with interspersed plains and varied coastlines—has significantly influenced its history, culture, and economy.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='고마워', additional_kwargs={},

In [ ]:
#데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session3_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session3_s
  Type: HUMAN
  Content: 한국의 기후에 대해 설명해줘
----------------------------------------
ID: 2, Session: session3_s
  Type: AI
  Content: 안녕하세요! 한국의 기후에 대해 설명해 드릴게요.  간단히 말씀드리면 한국은 **온대 계절풍 기후**에 속합니다.  하지만 지역에 따라 기후 차이가 상당히 크다는 점을 꼭 기억해주세요.  자세히 설명드리면 다음과 같습니다.

**1. 계절의 변화가 뚜렷한 온대 기후:** 사계절이 뚜렷하게 나타납니다. 봄에는 따뜻하고 습하며, 여름에는 덥고 습하고, 가을에는 시원하고 건조하며, 겨울에는 춥고 건조합니다.  특히, 여름과 겨울의 기온 차이가 매우 큽니다.

**2. 계절풍의 영향:**  계절풍의 영향을 크게 받습니다. 여름에는 남쪽에서 습한 해양성 계절풍이 불어와 무덥고 비가 많이 오며, 겨울에는 북쪽에서 차고 건조한 대륙성 계절풍이 불어와 춥고 건조합니다.  이 계절풍의 영향으로 여름에는 장마철이, 겨울에는 건조한 날씨가 지속됩니다.

**3. 지역별 기후 차이:**  한국의 지형적 특징 때문에 지역별 기후 차이가 매우 큽니다.
    * **서해안:** 여름철에 습도가 높고 강수량이 많으며, 겨울철에는 서해안에 위치한 섬들의 영향으로 눈이 적게 내립니다.  또한, 해풍의 영향으로 기온 변화가 완만합니다.
    * **동해안:** 겨울철에 강한 북서풍의 영향으로 강설량이 많고, 여름철에는 동해안을 따라 흐르는 냉수대의 영향으로 서해안보다 기온이 낮습니다.
    * **내륙:** 여름에는 낮과 밤의 기온 차가 크고, 겨울에는 매우 춥습니다. 강수량은 서해안이나 동해안보다 적습니다.
    * **남부지방:**  온화한 기후를 보이며, 

In [ ]:
#세션1. 메모리는 초기화됐지만 데이터베이스에 저장된 내용을 바탕으로 잘 대답함.
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session1_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 이번엔 지형에 대해 설명해줄래?

--- history에 저장된 내용 ---
[]
---history 내용 출력 끝---

챗봇 (답변 생성 중)...
테디님, 서울의 지형은 크게 북쪽의 북한산과 도봉산 등의 높은 산지와 남쪽의 한강 유역의 낮은 평야로 나눌 수 있습니다.  북쪽 산지는 서울의 자연적인 경계를 이루며,  남쪽으로 갈수록 지형이 완만해져 한강을 중심으로 넓은 평야가 형성되어 있습니다.  한강은 서울을 동서로 가르며,  도시 발전에 큰 영향을 미쳤습니다.  평야 지역은 도시 개발이 활발하게 이루어진 곳이며, 산지 지역은 주택가와 공원 등으로 이용되고 있습니다.  전반적으로 서울은 산과 평야가 어우러진 지형적 특징을 가지고 있으며, 이러한 지형적 특성이 서울의 도시 발전과 생활 양식에 큰 영향을 주고 있습니다.  더 자세한 부분이 궁금하시면 질문해주세요!


--- Summarizer Memory의 요약이 업데이트되었습니다. ---
입력: 고마워

--- history에 저장된 내용 ---
[SystemMessage(content="The AI describes the geography of Seoul as consisting primarily of mountainous terrain in the north (Bukhansan and Dobongsan mountains) and a lower, flatter plain in the south along the Han River.  The northern mountains form a natural boundary, while the terrain gradually slopes southward into the extensive plain cent

##SQLChatMessageHistory 직접 사용

In [ ]:
def chatbot(session_id):
  print("\n--- 대화형 챗봇 시작 ---")
  print("종료하려면 'exit' 또는 'quit'을 입력하세요.")
  print("챗봇: 안녕하세요. 무엇을 도와드릴까요?")
  sql_chat_history = get_chat_history(session_id)
  memory = get_session_summarizer(session_id)
  while True:
    user_input = input("입력: ")
    if user_input.lower() in ["exit", "quit"]:
        print("챗봇을 종료합니다.")
        break
    print("챗봇 (답변 생성 중)...")
    try:
      history = memory.load_memory_variables({})['history']
      print("\n--- history에 저장된 내용 ---")
      print(history)
      print("---history 내용 출력 끝---\n")
      inputs = {"input": user_input, 'history':history}

      chain = prompt | llm
      response = chain.invoke(inputs).content
      print(response)

      # 사용자 입력과 AI의 답변을 ConversationSummaryMemory에 전달하여 요약 업데이트
      memory.save_context(
                {"input": user_input}, # 사용자 입력
                {"output": response} # AI의 답변
      )

      # 사용자 메시지 추가
      sql_chat_history.add_user_message(
        user_input
      )
      # AI 메시지 추가
      sql_chat_history.add_ai_message(response)
      print("\n--- Summarizer Memory의 요약이 업데이트되었습니다. ---")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

In [ ]:
#세션2. 이전 대화와 이어지지 않음
session_id = 'session2_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
새로운 세션 ID: session2_s 에 대한 ConversationSummaryBufferMemory (요약용) 생성
입력: 이번엔 지형에 대해 설명해줄래?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[]
---history 내용 출력 끝---

네, 지형에 대해 설명해 드릴게요!  지형은 지표면의 높낮이와 경사, 그리고 그곳을 구성하는 물질들의 모양과 배열을 통틀어 말하는 거예요.  쉽게 말해, 우리 눈에 보이는 땅의 모습이라고 생각하시면 됩니다.  산, 계곡, 평야, 사막, 해안 등 다양한 모습으로 나타나죠.

지형은 크게 다음과 같은 요소들로 이루어져 있어요:

* **고도 (Elevation):** 해수면으로부터의 높이를 나타내요. 산 정상은 고도가 높고, 해구는 고도가 낮죠.
* **경사 (Slope):** 지표면의 기울기를 나타내요.  가파른 경사는 산악 지형을, 완만한 경사는 평야 지형을 나타내죠.
* **지형 형태 (Landform):** 산, 언덕, 계곡, 평야, 협곡, 봉우리, 능선, 분지 등 지표면의 구체적인 모양을 말해요.  이 형태들은 지각 변동, 침식, 퇴적 작용 등 다양한 과정을 통해 만들어집니다.
* **지질 (Geology):** 지형을 구성하는 암석의 종류와 구조를 말해요. 화강암, 현무암, 사암 등 다양한 암석이 지형의 특징을 결정하는 데 중요한 역할을 합니다.
* **토양 (Soil):** 지표면을 덮고 있는 토양의 종류와 특성도 지형에 영향을 미쳐요. 토양의 종류에 따라 식생도 달라지고, 그 결과 지형의 모습도 바뀌게 되죠.


지형은 만들어지는 과정에 따라 다음과 같이 분류하기도 해요:

* **내생적 지형:** 지구 내부의 힘(판구조론적 활동, 화산 활동 등)에 의해 만들어진 지형으로, 산맥, 화산, 협곡 등이 여기에 속해요.
* **외생적 지형:** 지구 

In [ ]:
#메모리 확인
summarizer_memory = get_session_summarizer(session_id)
summarizer_memory.load_memory_variables({})['history']

[SystemMessage(content="The human asks the AI to explain topography. The AI responds by defining topography as the variations in elevation, slope, and the materials composing the Earth's surface, encompassing features like mountains, valleys, plains, deserts, and coastlines.  The AI details key components: elevation (height above sea level), slope (steepness), landforms (specific shapes like mountains and valleys), geology (rock types and structures), and soil types.  It further categorizes topography into endogenic landforms (created by internal forces like plate tectonics and volcanism) and exogenic landforms (formed by external forces like weathering, erosion, and deposition).  Finally, the AI notes that topography is constantly changing due to climate, vegetation, and human activity, offering to provide more detail on specific landforms if requested.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='고마워.', additional_kwargs={}, response_metadata={}),
 AIMessage(

In [ ]:
#데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session2_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session2_s
  Type: HUMAN
  Content: 내 이름을 기억해?
----------------------------------------
ID: 2, Session: session2_s
  Type: AI
  Content: 아니요, 저는 대화형 인공지능 모델이기 때문에, 이전 대화를 기억하지 못합니다.  처음 만난 것처럼 생각해 주세요.  혹시 이름을 알려주시겠어요? 😊
----------------------------------------
ID: 3, Session: session2_s
  Type: HUMAN
  Content: 내 이름은 디테야
----------------------------------------
ID: 4, Session: session2_s
  Type: AI
  Content: 반갑습니다, 디테님!  무엇을 도와드릴까요?  😊
----------------------------------------
ID: 5, Session: session2_s
  Type: HUMAN
  Content: 제주도에 대해 간략하게 알려줄래?
----------------------------------------
ID: 6, Session: session2_s
  Type: AI
  Content: 제주도는 대한민국 남쪽에 위치한 화산섬으로, 아름다운 자연경관과 독특한 문화를 자랑하는 곳입니다.  한라산이라는 휴화산을 중심으로 섬 전체가 유네스코 세계자연유산으로 등재되어 있으며,  다양한 볼거리와 즐길 거리가 풍부합니다.  

주요 특징으로는:

* **아름다운 자연:** 푸른 바다, 검은 모래 해변, 울창한 숲, 그리고 한라산의 장엄한 풍경 등 다채로운 자연을 경험할 수 있습니다.
* **다양한 관광지:** 성산일출봉, 만장굴, 섭지코지, 정방폭포

In [ ]:
#세션1. 이전 대화를 기억하지 못함
session_id = 'session1_s'
chatbot(session_id)


--- 대화형 챗봇 시작 ---
종료하려면 'exit' 또는 'quit'을 입력하세요.
챗봇: 안녕하세요. 무엇을 도와드릴까요?
입력: 버스의 종류에 대해서 설명해줄래?
챗봇 (답변 생성 중)...

--- history에 저장된 내용 ---
[SystemMessage(content="The AI describes the geography of Seoul as consisting primarily of mountainous terrain in the north (Bukhansan and Dobongsan mountains) and a lower, flatter plain in the south along the Han River.  The northern mountains form a natural boundary, while the terrain gradually slopes southward into the extensive plain centered on the Han River, which significantly influenced the city's development.  The plain area is heavily developed, while the mountainous areas are used for housing and parks.  The AI concludes that Seoul's blend of mountains and plains has greatly impacted its development and lifestyle.", additional_kwargs={}, response_metadata={}), HumanMessage(content='고마워', additional_kwargs={}, response_metadata={}), AIMessage(content='천만에요, 테디님!  다른 궁금한 점이 있으면 언제든지 물어보세요.  즐거운 하루 보내세요!', additional_kwargs={}, response_metada

In [ ]:
#메모리 확인
memory = get_session_summarizer(session_id)
memory.load_memory_variables({})['history']

[SystemMessage(content="The AI describes the geography of Seoul as consisting primarily of mountainous terrain in the north (Bukhansan and Dobongsan mountains) and a lower, flatter plain in the south along the Han River. The northern mountains form a natural boundary, while the terrain gradually slopes southward into the extensive plain centered on the Han River, which significantly influenced the city's development. The plain area is heavily developed, while the mountainous areas are used for housing and parks.  The AI concludes that Seoul's blend of mountains and plains has greatly impacted its development and lifestyle.  Following a thank you from the human, the AI then describes the various types of buses in Seoul, categorizing them into city buses (including regular, trunk, branch, express, village, and metropolitan buses), intercity buses, and express buses.  The AI also mentions other specialized buses like tour buses and shuttles, noting that variations exist based on region an

In [ ]:
#데이터베이스 확인
get_history_from_db(session_id)

데이터베이스 '/content/drive/MyDrive/LangChain/session1_s_chat.db'에 성공적으로 연결되었습니다.

대화 기록
ID: 1, Session: session1_s
  Type: HUMAN
  Content: 서울에 대해 간략하게 설명해줘
----------------------------------------
ID: 2, Session: session1_s
  Type: AI
  Content: 서울은 대한민국의 수도이자 최대 도시로, 역사와 현대가 공존하는 매력적인 도시입니다.  한강을 중심으로 발전해왔으며, 고궁과 같은 전통 건축물부터 초고층 빌딩과 최첨단 시설까지 다양한 모습을 볼 수 있습니다.  세계적인 문화, 예술, 경제 중심지이기도 하며, 풍부한 역사 유적과 활기찬 현대 문화를 동시에 경험할 수 있는 곳입니다.  짧게 말해, 서울은 역동적이고 다채로운 매력을 가진 도시입니다.
----------------------------------------
ID: 3, Session: session1_s
  Type: HUMAN
  Content: 내 이름은 테디야
----------------------------------------
ID: 4, Session: session1_s
  Type: AI
  Content: 반갑습니다, 테디님!  무엇을 도와드릴까요?
----------------------------------------
ID: 5, Session: session1_s
  Type: HUMAN
  Content: 내 이름을 기억해?
----------------------------------------
ID: 6, Session: session1_s
  Type: AI
  Content: 네, 테디님.  이름 기억하고 있습니다!  무엇을 도와드릴까요?
----------------------------------------
ID: 7, Session: session1_s
  Type: HUMAN
  Co